In [1]:
import numpy as np
import open3d as o3d
import correct_lidar

import glob

import altair as alt
alt.data_transformers.enable('json')
import seaborn as sns
import matplotlib.pyplot as plt 
import pandas as pd
import datetime as dt
from sublimpy import utils
import act
import xarray as xr

# Open up binned lidar data

In [2]:
src_df = pd.read_parquet("binned_point_counts_1712641662.790818.parquet")
src_df = utils.modify_df_timezone(src_df, 'UTC', 'US/Mountain')
src_df['minutes'] = src_df['time'].dt.minute
src_df['hours'] = src_df['time'].dt.hour
src_df['date'] = src_df.time.dt.date

# Open up SOS tidy data

In [3]:
# Open SOS Measurement Dataset
################################################
start_date = '20221130'
end_date = '20230509'
# open files
tidy_df = pd.read_parquet(f'../paper1/tidy_df_{start_date}_{end_date}_noplanar_fit.parquet')

# convert time column to datetime
tidy_df['time'] = pd.to_datetime(tidy_df['time'])
tidy_df = utils.modify_df_timezone(tidy_df, 'UTC', 'US/Mountain')

# Download and open SAIL Disdrometer data

In [4]:
dates = src_df.time.dt.strftime('%Y-%m-%d').unique()

In [5]:
username = os.getenv("ARM_USERNAME")
token = os.getenv("ARM_TOKEN")

ecor_kp = 'guc30ecorS3.b1'
sebs = 'gucsebsS3.b1'
ld = 'gucldM1.b1'
output_dir = os.path.join('/Users/elischwat/Development/data/sublimationofsnow', ld)

In [6]:
files = []
for date in dates:
    new_files = act.discovery.download_arm_data(
        username,
        token,
        ld,
        date,
        date,
        output=output_dir
    )
    files = files + new_files
disdro_ds = xr.open_mfdataset(files)

[DOWNLOADING] gucldM1.b1.20221211.000000.cdf

If you use these data to prepare a publication, please cite:

Wang, D., Zhu, Z., & Shi, Y. Laser Disdrometer (LD). Atmospheric Radiation
Measurement (ARM) User Facility. https://doi.org/10.5439/1973058

[DOWNLOADING] gucldM1.b1.20221212.000000.cdf

If you use these data to prepare a publication, please cite:

Wang, D., Zhu, Z., & Shi, Y. Laser Disdrometer (LD). Atmospheric Radiation
Measurement (ARM) User Facility. https://doi.org/10.5439/1973058

[DOWNLOADING] gucldM1.b1.20221213.000000.cdf

If you use these data to prepare a publication, please cite:

Wang, D., Zhu, Z., & Shi, Y. Laser Disdrometer (LD). Atmospheric Radiation
Measurement (ARM) User Facility. https://doi.org/10.5439/1973058

[DOWNLOADING] gucldM1.b1.20221214.000000.cdf

If you use these data to prepare a publication, please cite:

Wang, D., Zhu, Z., & Shi, Y. Laser Disdrometer (LD). Atmospheric Radiation
Measurement (ARM) User Facility. https://doi.org/10.5439/1973058

[DOW

In [7]:
disdro_ds = utils.modify_xarray_timezone(disdro_ds, 'UTC', 'US/Mountain')
disdro_ds

<xarray.Dataset>
Dimensions:                               (time: 24480, particle_size: 32,
                                           raw_fall_velocity: 32)
Coordinates:
  * particle_size                         (particle_size) float32 0.062 ... 24.0
  * raw_fall_velocity                     (raw_fall_velocity) float32 0.05 .....
    time (US/Mountain)                    (time) datetime64[ns] 2022-12-10T17...
    time (UTC)                            (time) datetime64[ns] 2022-12-11 .....
  * time                                  (time) datetime64[ns] 2022-12-10T17...
Data variables: (12/42)
    base_time                             (time) datetime64[ns] 2022-12-11 .....
    time_offset                           (time) datetime64[ns] dask.array<chunksize=(1440,), meta=np.ndarray>
    precip_rate                           (time) float32 dask.array<chunksize=(1440,), meta=np.ndarray>
    qc_precip_rate                        (time) int32 dask.array<chunksize=(1440,), meta=np.ndarray>
    weather_code                          (time) float32 dask.array<chunksize=(1440,), meta=np.ndarray>
    qc_weather_code                       (time) int32 dask.array<chunksize=(1440,), meta=np.ndarray>
    ...                                    ...
    moment4                               (time) float32 dask.array<chunksize=(1440,), meta=np.ndarray>
    moment5                               (time) float32 dask.array<chunksize=(1440,), meta=np.ndarray>
    moment6                               (time) float32 dask.array<chunksize=(1440,), meta=np.ndarray>
    lat                                   (time) float32 38.96 38.96 ... 38.96
    lon                                   (time) float32 -107.0 ... -107.0
    alt                                   (time) float32 2.886e+03 ... 2.886e+03
Attributes: (12/15)
    command_line:                ld_ingest -s guc -f M1
    process_version:             ingest-ld-1.4-0.el7
    dod_version:                 ld-b1-2.0
    Conventions:                 ARM-1.2
    data_level:                  b1
    platform_id:                 ld
    ...                          ...
    facility_id:                 M1
    sampling_interval:           1 minute
    effective_measurement_area:  0.54 m^2
    doi:                         10.5439/1560674
    datastream:                  gucldM1.b1
    history:                     created by user dsmgr on machine prod-proc4....

In [8]:
src_df = pd.read_parquet("binned_point_counts_1712641662.790818.parquet")
src_df = utils.modify_df_timezone(src_df, 'UTC', 'US/Mountain')
src_df['minutes'] = src_df['time'].dt.minute
src_df['hours'] = src_df['time'].dt.hour
src_df['date'] = src_df.time.dt.date

# Look at binned profiles

In [9]:
src = src_df[src_df.time.dt.strftime('%Y-%m-%d') == '2022-12-12'].query("z_bin > -10")
(
    alt.Chart(
        src
    ).mark_line().encode(
        alt.X("count:Q").scale(type='symlog').sort('-y').axis(values=[0, 1,100,10000,10000]),
        alt.Y("z_bin:Q"),
        alt.Color("minutes:O"),
        alt.Facet("hours:O", columns=9)
    ).properties(width = 100, height = 100) &\
    alt.Chart(
        tidy_df[tidy_df.variable.isin(['SF_avg_1m_ue', 'SF_avg_2m_ue'])][
            (tidy_df.time >= src.time.min())
            &
            (tidy_df.time <= src.time.max())
        ]
    ).mark_line().encode(
        alt.X("time:T"),
        alt.Y("value:Q"),    
        alt.Color("height:N")
).properties(width = 1100, height = 100)
).resolve_scale(color='independent')

/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_64605/2071590756.py:12: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  tidy_df[tidy_df.variable.isin(['SF_avg_1m_ue', 'SF_avg_2m_ue'])][


alt.VConcatChart(...)

#### Calculate the profiles used to remove physical obstructions

In [10]:
profile_df = src_df.query("z_bin > -8")

In [11]:
obstruction_normalize = profile_df[
    (profile_df['time'] >= "2022-12-11 17:00:00")
    &
    (profile_df['time'] < "2022-12-12 05:00:00")
]
obstruction_normalize = obstruction_normalize.groupby('z_bin')[['count']].median()
obstruction_normalize

,count
z_bin,
-7.95,1347.5
-7.85,1004.5
-7.75,986.0
-7.65,1002.0
-7.55,977.0
...,...
0.55,0.0
0.65,0.0
0.75,1.0


#### Remove them

In [12]:
physical_obstructions_removed = profile_df.groupby(["time", "hours", "minutes", "date"]).apply(lambda df: df.set_index('z_bin') - obstruction_normalize).drop(columns=['time', 'hours', 'minutes', 'date']).reset_index()
physical_obstructions_removed

/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_64605/1090000286.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  physical_obstructions_removed = profile_df.groupby(["time", "hours", "minutes", "date"]).apply(lambda df: df.set_index('z_bin') - obstruction_normalize).drop(columns=['time', 'hours', 'minutes', 'date']).reset_index()


,time,hours,minutes,date,z_bin,count
0,2022-12-11 17:00:00,17,0,2022-12-11,-7.95,-19.5
1,2022-12-11 17:00:00,17,0,2022-12-11,-7.85,1.5
2,2022-12-11 17:00:00,17,0,2022-12-11,-7.75,41.0
3,2022-12-11 17:00:00,17,0,2022-12-11,-7.65,13.0
4,2022-12-11 17:00:00,17,0,2022-12-11,-7.55,-12.0
...,...,...,...,...,...,...
311035,2023-01-09 16:55:00,16,55,2023-01-09,0.55,214.0
311036,2023-01-09 16:55:00,16,55,2023-01-09,0.65,41.0
311037,2023-01-09 16:55:00,16,55,2023-01-09,0.75,10.0
311038,2023-01-09 16:55:00,16,55,2023-01-09,0.85,0.0


In [13]:
src = physical_obstructions_removed[physical_obstructions_removed.time.dt.strftime('%Y-%m-%d') == '2022-12-12'].query("z_bin > -10")
alt.Chart(
    src
).mark_line().encode(
    alt.X("count:Q").scale(type='linear').sort('-y'),
    alt.Y("z_bin:Q"),
    alt.Color("minutes:O"),
    alt.Facet("hours:O", columns=6)
).properties(width = 100, height = 100)

alt.Chart(...)

#### Vertically integrate point concentrations

In [14]:
physical_obstructions_removed_integrated = physical_obstructions_removed.groupby('time')[['count']].sum().reset_index()

#### Vertically integrate point concentrations

In [15]:
physical_obstructions_removed_integrated = physical_obstructions_removed.groupby('time')[['count']].sum().reset_index()

# Plot case studies

In [16]:
def case_study_chart(start_date, end_date):
    src_prof = physical_obstructions_removed_integrated[
        (physical_obstructions_removed_integrated.time >= start_date)
        & 
        (physical_obstructions_removed_integrated.time <= end_date)
    ]
    src_tidy = tidy_df[
        (tidy_df.time >= start_date)
        & 
        (tidy_df.time <= end_date)
    ]

    src_disdro = disdro_ds[['precip_rate']].to_dataframe().reset_index()
    src_disdro = src_disdro[
        (src_disdro.time >= start_date)
        & 
        (src_disdro.time <= end_date)
    ]
    return (
        alt.Chart(
            src_disdro
        ).mark_line().encode(
            alt.X("time:T"),
            alt.Y("precip_rate:Q").title(["Disdrometer", "Precip rate (mm)"])
        ).properties(height = 100)
        &\
        # alt.Chart(
        #     src_prof
        # ).mark_line().encode(
        #     alt.X("time:T"),
        #     alt.Y("count:Q").title(["Vertically-integrated", "point count"]).scale(type='symlog')
        # ).properties(height = 100)
        # &\
        alt.Chart(
            src_tidy[src_tidy.variable.isin(['SF_avg_1m_ue', 'SF_avg_2m_ue'])]
        ).mark_line().encode(
            alt.X("time:T"),
            alt.Y("value:Q").title(["FlowCapt blowing", "snow flux (g/m^2/s)"]).scale(
                # type='symlog'
            ),
            alt.Color("height:N")
        ).properties(height = 100) &\
        alt.Chart(
            src_tidy[src_tidy.variable.isin(['ldiag_10m_ue', 'ldiag_10m_uw', 'ldiag_10m_c', 'ldiag_10m_c'])]
        ).mark_line().encode(
            alt.X("time:T"),
            alt.Y("value:Q").title(["Sonic anemometer", "quality flag"]),
            alt.Color("tower:N")
        ).properties(height = 100) &\
        alt.Chart(
            src_tidy[src_tidy.variable.isin(['w_h2o__10m_ue', 'w_h2o__10m_uw', 'w_h2o__10m_c', 'w_h2o__10m_c'])]
        ).mark_line().encode(
            alt.X("time:T"),
            alt.Y("value:Q").title("Water vapor flux (g/m^2/s)"),    
            alt.Color("tower:N")
        ).properties(height = 100) &\
        (
            alt.Chart(
                src_tidy[src_tidy.variable.isin(['spd_10m_c'])]
            ).mark_line(color='red').encode(
                alt.X("time:T"),
                alt.Y("value:Q").title("(red) 10m Wind speed (m/s)")
            ).properties(height = 100) +
            alt.Chart(
                src_tidy[src_tidy.variable.isin(['RH_10m_c'])]
            ).mark_line(color='blue').encode(
                alt.X("time:T"),
                alt.Y("value:Q").title("(blue) 10m RH (%)")
            ).properties(height = 100)
        ).resolve_scale(y='independent')
    ).resolve_scale(color='independent', x = 'shared')

In [17]:
case_study_chart("20221212 1200", "20221214") |\
case_study_chart("20221220", "20221222") |\
case_study_chart('20221230', '20230103') |\
case_study_chart('20230105', '20230109')

alt.HConcatChart(...)